# Semi-Supervised Label Transfer of RGC Subtype Annotations from Liang et. al. 2023 to Day 21 RGC-iNs

In [ ]:
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import scvi
import anndata as ad

sc.settings.verbosity = 3 # hints

### Data Collection

In [ ]:
### Read data into an AnnData object (this step may take a few minutes the first time)

# RGC subset obtained from
# https://cellxgene.cziscience.com/collections/af893e86-8e9f-41f1-a474-ef05359b1fb7
adata_liang = ad.read_h5ad('./../../sc_data/liang_adult_rgcs/local.h5ad')

# NAIP2 RGC-iNs
adata_21 = sc.read_10x_mtx(
    './../../sc_data/d21/',
    var_names='gene_symbols',
    cache=True
)

adata_liang.var_names_make_unique()
adata_21.var_names_make_unique()
display(adata_21, adata_liang)

In [ ]:
# Add 'sample' column in order to later distinguish cells from different experiments
adata_liang.obs['sample'] = 'liang'
adata_21.obs['sample'] = 'd21'

### Preprocessing of Day 21 RGC-iN Dataset

In [ ]:
sc.pp.filter_cells(adata_21, min_genes=800)
sc.pp.filter_genes(adata_21, min_cells=5)

# annotate the group of mitochondrial genes as 'mt' for later removal
adata_21.var['mt'] = adata_21.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'

# generate plots to assess cell/transcript quality
sc.pp.calculate_qc_metrics(adata_21, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata_21, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata_21, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_21, x='total_counts', y='n_genes_by_counts')

In [ ]:
# filter out cells with an abnormal number of total_counts and those with high mitochondrial gene presence
adata_21 = adata_21[adata_21.obs.total_counts < 150000, :]
adata_21 = adata_21[adata_21.obs.pct_counts_mt < 5, :]

# normalize and logarthmize data
sc.pp.normalize_total(adata_21, target_sum=1e4)
sc.pp.log1p(adata_21)

display(adata_21)

In [ ]:
# add 'author_cell_type' column to match Liang dataset
adata_21.obs['author_cell_type'] = 'unknown'
adata_21.obs

### Preprocessing of Combined Liang et. al. 2023 / Day 21 RGC-iN Dataset

In [ ]:
# concatenate Day 21 NAIP2 data and Liang data into one dataframe
adata = ad.concat([adata_liang, adata_21])

# normalize and logarthmize the concatenated dataframe
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

# use only highly variable genes
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=2000, layer='counts', batch_key='sample', subset=True)

display(adata)

### Label Transfer

In [ ]:
# setup and train a scvi model based on our concatenated data (can take some time, especially without a GPU)
scvi.model.SCVI.setup_anndata(adata, layer = 'counts', batch_key='sample')
vae = scvi.model.SCVI(adata)
vae.train()

In [ ]:
# perform label transfer (can take some time, especially without a GPU)
lvae = scvi.model.SCANVI.from_scvi_model(vae, adata=adata, unlabeled_category='unknown', labels_key='author_cell_type')
lvae.train(max_epochs=20, n_samples_per_label=100)

In [ ]:
# add a 'predicted' column to our dataframe denoting the label transfer annotations
adata.obs['predicted'] = lvae.predict(adata)
adata.obs

In [ ]:
# extract the Day 21 samples from our concatenated dataframe (these will have predicted labels)
adata_21_subset = adata[adata.obs["sample"] == "d21"]

# create a dictionary mapping each Day 21 barcode (each cell) to its predicted cell type
cell_mapper = dict(zip(adata_21_subset.obs.index, adata_21_subset.obs.predicted))

# use dictionary to map predicted cell types back onto the original Day 21 dataframe
# this dataframe was never normalized with the Liang data
adata_21.obs['predicted_cell_type'] = adata_21.obs.index.map(cell_mapper)
adata_21.obs

### Clustering and Dimensionality Reduction of Day 21 RGC-iNs with Transferred Labels

In [ ]:
# begin by subsetting dataframe on top 2000 highly variable genes
adata_21.layers["counts"] = adata_21.X.copy()
sc.pp.highly_variable_genes(adata_21, flavor='seurat_v3', n_top_genes=2000, subset = True, layer='counts')

In [ ]:
sc.tl.pca(adata_21, svd_solver='arpack', use_highly_variable=False)
sc.pl.pca(adata_21)
sc.pl.pca_variance_ratio(adata_21, log=True)

In [ ]:
sc.pp.neighbors(adata_21)
sc.tl.louvain(adata_21)
sc.tl.umap(adata_21)

### UMAP Visualization of All Datasets

In [ ]:
sc.pl.umap(adata_21, color='louvain')
sc.pl.umap(adata_21, color='predicted_cell_type')

In [ ]:
### Condense predicted cell types (ON-MGC + OFF-MGS => MGC, etc.)

MGC = list(adata_21.obs['predicted_cell_type'])
for i, label in enumerate(MGC):
    MGC[i] = 'True' if 'MGC' in label else 'False'
    
print(f'MGC percentage is {MGC.count("True")/len(MGC)}')

PGC = list(adata_21.obs['predicted_cell_type'])
for i, label in enumerate(PGC):
    PGC[i] = 'True' if 'PGC' in label else 'False'
    
print(f'PGC percentage is {PGC.count("True")/len(PGC)}')

OPN = list(adata_21.obs['predicted_cell_type'])
for i, label in enumerate(OPN):
    OPN[i] = 'True' if 'OPN' in label else 'False'
    
print(f'OPN percentage is {OPN.count("True")/len(OPN)}')

UNCLASSIFIED = list(adata_21.obs['predicted_cell_type'])
for i, label in enumerate(UNCLASSIFIED):
    if label == 'RGC6' or label == 'RGC7' or label == 'RGC8':
        UNCLASSIFIED[i] = 'True'
    else:
        UNCLASSIFIED[i] = 'False'
    
print(f'UNCLASSIFIED percentage is {UNCLASSIFIED.count("True")/len(UNCLASSIFIED)}')

# Calculate population percentages of different cell types and plot on a pie chart
print(f'Total: {MGC.count("True")/len(MGC) + PGC.count("True")/len(PGC) + OPN.count("True")/len(OPN) + UNCLASSIFIED.count("True")/len(UNCLASSIFIED)}')
pie_val = [MGC.count("True")/len(MGC), PGC.count("True")/len(PGC), OPN.count("True")/len(OPN), UNCLASSIFIED.count("True")/len(UNCLASSIFIED)]
pie_lab = ['MGCs', 'PGCs', 'ipRGCs', 'Uncategorized RGCs']
plt.pie(pie_val, labels = pie_lab, colors=colors, autopct='%.0f%%')
plt.savefig('figures/pie.pdf', dpi=400)

COMBINED = list(adata_21.obs['predicted_cell_type'])
for i, label in enumerate(COMBINED):
    if 'MGC' in label:
        COMBINED[i] = 'MGC'
    elif 'PGC' in label:
        COMBINED[i] = 'PGC'
    elif 'OPN' in label:
        COMBINED[i] = 'OPN'
    else:
        COMBINED[i] = 'UNCATEGORIZED'

adata_21.obs['MGC'] = MGC
adata_21.obs['PGC'] = PGC
adata_21.obs['OPN'] = OPN
adata_21.obs['UNCLASSIFIED'] = UNCLASSIFIED
adata_21.obs['condensed_cell_types'] = COMBINED

In [ ]:
pallette = {
    'True':'darkblue',
    'False':'lightgray'
}

colors = sns.color_palette('deep')[0:4]

sc.pl.umap(adata_21, color='louvain')
sc.pl.umap(adata_21, color='condensed_cell_types', palette=colors, save='_d21_subtypes.pdf')
sc.pl.umap(adata_21, color='MGC', palette=pallette)
sc.pl.umap(adata_21, color='PGC', palette=pallette)
sc.pl.umap(adata_21, color='OPN', palette=pallette)
sc.pl.umap(adata_21, color='UNCLASSIFIED', palette=pallette)

In [ ]:
# Visualize Liang et. al. 2023 data with their own subtype annotations
sc.pl.umap(adata_liang, color=['author_cell_type'], legend_loc='on data', save='_liang_subtypes.pdf')

Written by Manan Chopra (m1chopra@ucsd.edu) @ Wahlin Lab  
Last updated 5/26/2023